In [1]:
# # This cell runs bowtie locally, uncomment last line to run the command and test your bowtie
# 
# import os
# # wget index and SRR043378 directly into project folder
# threads = 4  # Can be changed
# # folder name then file name
# bowtieIndex = "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index/" \
#               "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"
# # fastq read file


# reads = "SRR043378_1.fastq/data"
# cmd = "bowtie2 -p "+str(threads)+" -x " + bowtieIndex + " -U "+reads+" -S out.sam"
# 
# 
# os.system(cmd)

In [2]:
# import pyspark
# import os
# 
# # new sparkContext
# sc = pyspark.SparkContext(appName="DNA")

In [3]:
# Basis for map function

# threads = 1  # Can be changed
# # folder name then file name
# bowtieIndex = "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index/" \
#               "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"
# # fastq read file
# reads = "SRR043378_1.fastq/data"
# cmd = "bowtie2 -p " + str(threads) + " -x " + bowtieIndex + " -U " + reads + " -S out.sam"
# os.system(cmd)

In [1]:
#import findspark
#findspark.init('/home/taylor/spark')

from pyspark import SparkConf, SparkContext, TaskContext
import math
from pyspark.sql import SQLContext


conf = SparkConf().setMaster("local").setAppName("DNA")

sc = SparkContext(conf=conf)

In [3]:
# creates tuples with format (string, line number)
raw_input = (sc.textFile("SRR043378_1.fastq/data")).zipWithIndex()

In [5]:
# create tuple with format (read number, string)
indexed_raw_input = raw_input.map(lambda x: (math.floor(x[1]/4), x[0]))

In [7]:
# Function to pass to map function that concatenates the 4 strings
def make_reads(iterable_read):
    x = 0
    line1 = ""
    line2 = ""
    line3 = ""
    line4 = ""
    for line in iterable_read:
        if x == 0:
            line1 = line
        if x == 1:
            line2 = line
        if x == 2:
            line3 = line
        if x == 3:
            line4 = line
        x = x+1
    return "%s\n%s\n%s\n%s" % (line1, line2, line3, line4)

In [9]:
reads_tuple = indexed_raw_input.groupByKey().mapValues(lambda x: make_reads(x))

In [11]:
# test = reads_tuple.take(10)

In [14]:
# Test get one record (brackets denote first tuple, second field because it is in format (0, read), (1, read) etc...)
# print(test[0][1])

In [16]:
# num_reads = reads_tuple.count()

In [19]:
# print("Number of reads: %s" % num_reads)
# 16,924,141 reads

In [11]:
# Sort by key, which is just the line number, and then get the values, which is just the read.
readsRDD = reads_tuple.sortByKey().values()

In [21]:
# test2 = readsRDD.take(10)

In [22]:
# print(test2[0])

In [ ]:
# readsRDD = readsRDD.repartition(10)

In [25]:
# readsRDD.mapPartitionsWithIndex()

In [ ]:
# readsRDD.saveAsTextFile('partitioned_data/')

In [13]:
bowtie_index = "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index/" \
               "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"

alignment_pipe = readsRDD.pipe("bowtie2 -x --no-sq " + bowtie_index + " -")

In [14]:
# # outputRDD = sc.emptyRDD()
# 
# 
# def reduce_to_sam(output):
#     try:
#         # print(type(output))
#         # tempRDD = sc.parallelize([output]).collect()
#         output.saveAsTextFile('output.sam')
#         # outputrdd = output.collect()  # .reduce(lambda x, y: str(x)+"\n"+str(y))
#     except Exception as ex:
#         print(ex)

In [19]:
# file = open("output.sam", 'r+b')
alignment_pipe.saveAsTextFile('output.sam')  # .foreach(lambda x: reduce_to_sam(x))  # .saveAsTextFile('output.sam') 
# file.close()

Py4JJavaError: An error occurred while calling o147.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/home/taylor/PycharmProjects/DNA_Project/output.sam already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:283)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache

In [22]:
# TODO: split data into batches and send to bowtie 
# https://github.com/citiususc/SparkBWA/blob/master/src/main/java/com/github/sparkbwa/BwaAlignmentBase.java


In [26]:
sc.stop()